In [1]:
! pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 12.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [dash]8/9 [dash]]gerous]


In [2]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html, Input, Output
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [4]:
pip install openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]1/2 [openpyxl]
Note: you may need to restart the kernel to use updated packages.


#### Load Dataset

In [5]:
file_path = "Provider Sample DataSet MRP 2025 Spring.xlsx"
df = pd.read_excel(file_path, sheet_name="MetricData")

#### Data Cleaning

In [6]:
df.columns = df.columns.str.strip().str.replace(" ", "_").str.lower()
df["reportingperiodstartdate"] = pd.to_datetime(df["reportingperiodstartdate"], errors="coerce")
df["reportingperiodenddate"] = pd.to_datetime(df["reportingperiodenddate"], errors="coerce")

# Drop missing values
df = df.dropna(subset=["value", "specialty"])

### Machine Learning - Clustering Physicians by Workload

In [ ]:
# Initialize Dash App
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Physician Burnout Analysis Dashboard"),

    # Dropdown for Specialty Selection
    html.Label("Select Specialty:"),
    dcc.Dropdown(
        id="specialty-dropdown",
        options=[{"label": spec, "value": spec} for spec in df["specialty"].unique()],
        value=df["specialty"].unique()[0],
        multi=False
    ),

    # Line Chart
    dcc.Graph(id="trend-graph"),

    # Scatter Plot
    dcc.Graph(id="cluster-graph")
])

# Callbacks to update charts
@app.callback(
    Output("trend-graph", "figure"),
    Input("specialty-dropdown", "value")
)
def update_trend_chart(specialty):
    df_filtered = df[df["specialty"] == specialty]
    fig = px.line(df_filtered, x="reportingperiodstartdate", y="value", title=f"Workload Trend: {specialty}")
    return fig

@app.callback(
    Output("cluster-graph", "figure"),
    Input("specialty-dropdown", "value")
)
def update_cluster_chart(specialty):
    df_filtered = df[df["specialty"] == specialty]
    fig = px.scatter(df_filtered, x="reportingperiodstartdate", y="value", color=df_filtered["cluster"].astype(str),
                     title=f"Clustering of Workload: {specialty}")
    return fig

# Run App
if __name__ == "__main__":
    app.run(debug=True)

[2025-07-16 23:40:22,534] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/flask/app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/dash/dash.py", line 1484, in dispatch
    response_data = ctx.run(partial_func)
                    ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/dash/_callback.py", line 698, in add_context
    raise err
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/dash/_callback.py", line 689, in add_context

In [8]:
scaler = StandardScaler()
df["scaled_value"] = scaler.fit_transform(df[["value"]])
kmeans = KMeans(n_clusters=3, random_state=42)
df["cluster"] = kmeans.fit_predict(df[["scaled_value"]])